# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Apr 19 2023 9:58AM,261395,10,MSP220912,"Methapharm, Inc.",Released
1,Apr 19 2023 9:58AM,261395,10,MSP220914,"Methapharm, Inc.",Released
2,Apr 19 2023 9:58AM,261395,10,MSP220915,"Methapharm, Inc.",Released
3,Apr 19 2023 9:58AM,261395,10,MSP220917,"Methapharm, Inc.",Released
4,Apr 19 2023 9:58AM,261395,10,MSP220918,"Methapharm, Inc.",Released
5,Apr 19 2023 9:55AM,261394,10,MSP220913,Methapharm-G,Released
6,Apr 19 2023 9:54AM,261393,10,7830244,Emerginnova,Released
7,Apr 19 2023 9:45AM,261321,15,PNC442554B,"Person & Covey, Inc.",Released
8,Apr 19 2023 9:45AM,261321,15,PNC442680A,"Person & Covey, Inc.",Released
9,Apr 19 2023 9:45AM,261321,15,PNC442681A,"Person & Covey, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
23,261390,Executing,1
24,261392,Released,1
25,261393,Released,1
26,261394,Released,1
27,261395,Released,5


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
261390,1.0,NaN
261392,NaN,1.0
261393,NaN,1.0
261394,NaN,1.0
261395,NaN,5.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
260869,3.0,30.0
260910,7.0,6.0
261117,9.0,1.0
261320,8.0,10.0
261321,0.0,25.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
260869,3,30
260910,7,6
261117,9,1
261320,8,10
261321,0,25


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,260869,3,30
1,260910,7,6
2,261117,9,1
3,261320,8,10
4,261321,0,25


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,260869,3,30
1,260910,7,6
2,261117,9,1
3,261320,8,10
4,261321,,25


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Apr 19 2023 9:58AM,261395,10,"Methapharm, Inc."
5,Apr 19 2023 9:55AM,261394,10,Methapharm-G
6,Apr 19 2023 9:54AM,261393,10,Emerginnova
7,Apr 19 2023 9:45AM,261321,15,"Person & Covey, Inc."
32,Apr 19 2023 9:45AM,261362,15,"Alliance Pharma, Inc."
33,Apr 19 2023 9:41AM,261366,15,"Alliance Pharma, Inc."
61,Apr 19 2023 9:31AM,261320,15,"Brookfield Pharmaceuticals, LLC"
79,Apr 19 2023 9:25AM,261392,10,"Citieffe, Inc."
80,Apr 19 2023 9:23AM,261390,19,"Emersa Waterbox, LLC"
81,Apr 19 2023 9:21AM,261388,16,Sartorius Bioprocess Solutions


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Apr 19 2023 9:58AM,261395,10,"Methapharm, Inc.",,5
1,Apr 19 2023 9:55AM,261394,10,Methapharm-G,,1
2,Apr 19 2023 9:54AM,261393,10,Emerginnova,,1
3,Apr 19 2023 9:45AM,261321,15,"Person & Covey, Inc.",,25
4,Apr 19 2023 9:45AM,261362,15,"Alliance Pharma, Inc.",,1
5,Apr 19 2023 9:41AM,261366,15,"Alliance Pharma, Inc.",,28
6,Apr 19 2023 9:31AM,261320,15,"Brookfield Pharmaceuticals, LLC",8,10
7,Apr 19 2023 9:25AM,261392,10,"Citieffe, Inc.",,1
8,Apr 19 2023 9:23AM,261390,19,"Emersa Waterbox, LLC",1,
9,Apr 19 2023 9:21AM,261388,16,Sartorius Bioprocess Solutions,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Apr 19 2023 9:58AM,261395,10,"Methapharm, Inc.",5,
1,Apr 19 2023 9:55AM,261394,10,Methapharm-G,1,
2,Apr 19 2023 9:54AM,261393,10,Emerginnova,1,
3,Apr 19 2023 9:45AM,261321,15,"Person & Covey, Inc.",25,
4,Apr 19 2023 9:45AM,261362,15,"Alliance Pharma, Inc.",1,
5,Apr 19 2023 9:41AM,261366,15,"Alliance Pharma, Inc.",28,
6,Apr 19 2023 9:31AM,261320,15,"Brookfield Pharmaceuticals, LLC",10,8
7,Apr 19 2023 9:25AM,261392,10,"Citieffe, Inc.",1,
8,Apr 19 2023 9:23AM,261390,19,"Emersa Waterbox, LLC",,1
9,Apr 19 2023 9:21AM,261388,16,Sartorius Bioprocess Solutions,1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Apr 19 2023 9:58AM,261395,10,"Methapharm, Inc.",5,
1,Apr 19 2023 9:55AM,261394,10,Methapharm-G,1,
2,Apr 19 2023 9:54AM,261393,10,Emerginnova,1,
3,Apr 19 2023 9:45AM,261321,15,"Person & Covey, Inc.",25,
4,Apr 19 2023 9:45AM,261362,15,"Alliance Pharma, Inc.",1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Apr 19 2023 9:58AM,261395,10,"Methapharm, Inc.",5.0,NaN
1,Apr 19 2023 9:55AM,261394,10,Methapharm-G,1.0,NaN
2,Apr 19 2023 9:54AM,261393,10,Emerginnova,1.0,NaN
3,Apr 19 2023 9:45AM,261321,15,"Person & Covey, Inc.",25.0,NaN
4,Apr 19 2023 9:45AM,261362,15,"Alliance Pharma, Inc.",1.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Apr 19 2023 9:58AM,261395,10,"Methapharm, Inc.",5.0,0.0
1,Apr 19 2023 9:55AM,261394,10,Methapharm-G,1.0,0.0
2,Apr 19 2023 9:54AM,261393,10,Emerginnova,1.0,0.0
3,Apr 19 2023 9:45AM,261321,15,"Person & Covey, Inc.",25.0,0.0
4,Apr 19 2023 9:45AM,261362,15,"Alliance Pharma, Inc.",1.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,2613793,28.0,10.0
15,1045369,64.0,8.0
16,261388,1.0,0.0
19,1306822,29.0,2.0
20,782896,37.0,19.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,2613793,28.0,10.0
1,15,1045369,64.0,8.0
2,16,261388,1.0,0.0
3,19,1306822,29.0,2.0
4,20,782896,37.0,19.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,28.0,10.0
1,15,64.0,8.0
2,16,1.0,0.0
3,19,29.0,2.0
4,20,37.0,19.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,28.0
1,15,Released,64.0
2,16,Released,1.0
3,19,Released,29.0
4,20,Released,37.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,19,20
Status,,,,,
Executing,10.0,8.0,0.0,2.0,19.0
Released,28.0,64.0,1.0,29.0,37.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,19,20
0,Executing,10.0,8.0,0.0,2.0,19.0
1,Released,28.0,64.0,1.0,29.0,37.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,19,20
0,Executing,10.0,8.0,0.0,2.0,19.0
1,Released,28.0,64.0,1.0,29.0,37.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()